## Homework 08:  Parallel Programming 01

## Due Date: Apr 20, 2020, 08:00am

#### Firstname Lastname: Chengwei Chen

#### E-mail: cc6576@nyu.edu

#### Enter your solutions and submit this notebook

---

**Problem 1 (50p)**

Write an MPI program `sol08pr01.py` that does the following for some arbitrary number of processes $N \geq 2$. Here the number of processes $N$ is given as `N` while calling the code `sol08pr01.py` as: 

`mpirun -n N python3 sol08pr01.py`


Every process will contain one buffer with one integer variable, each of which is initialized to $0$.

Consequently, for $r=0, 1, \dots, N - 1$, Process $r$ squares its rank $r$, adds the result $r^2$ to the value of its own buffer, and then sends the sum to Process $r + 1$. Note that for $r=N-1$ the result will be sent to Process $0$, i.e. by convention, Process $N$ is the same as Process $0$. At the end Process $0$ prints the received value. 

Provide results for: $N = 10$, $N = 15$, $N = 20$, $N = 25$.



**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code. 



In [100]:
%%writefile sol08pr01.py

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
data = 0 # initialized to 0

if rank == 0:
    comm.send(data, dest=1)
    data = comm.recv(source=size-1)
    print(data)
elif rank == size-1:
    data = comm.recv(source=rank-1)
    data = data + rank**2
    comm.send(data, dest=0)
else:
    data = comm.recv(source=rank-1)
    data = data + rank**2
    comm.send(data, dest=rank+1)

Overwriting sol08pr01.py


In [101]:
!mpirun -n 10 python3 sol08pr01.py

285


In [102]:
!mpirun -n 15 python3 sol08pr01.py

1015


In [103]:
!mpirun -n 20 python3 sol08pr01.py

2470


In [104]:
!mpirun -n 25 python3 sol08pr01.py

4900


---

**Problem 2 (50p)**

Write an MPI program that does the following. There are two processes 0 and 1 that have to exchange $T=10$ messages.  


Process 0 initially reads two float variables from the standard input, call them $x, y$, and must ensure $x \neq 0$ and $y \neq 0$. For example this can be done as:

```
import sys


for line in sys.stdin:
    x = float(line)        
    if x != 0.0:
        break
for line in sys.stdin:
    y = float(line)        
    if y != 0.0:
        break
```


Both Process 0 and Process 1 will carry main results in an element that is part of a process buffer and called $p$. The value in $p$ is initially set to $1$. 


Now the exchange of messages is as follows.


0. Message00: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

1. Message01: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

2. Message01: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

3. Message02: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.


etc.

8. Message08: Process 0 multiplies its own value in $p$ by $x$ and sends the whole buffer to Process 1.

9. Message09: Process 1 divides its own value in $p$ by $y$ and sends the whole buffer to Process 0.

Finally, Process 0 prints the value in $p$ as a final result. 


Write the code that implements the protocol above. Additionally, provide results for: $(x, y) = (2, 4)$, $(x, y) = (1, 3)$, $(x, y) = (5, 7)$ and $(x, y) = (5, 10)$.


**Note**: You can use either blocking or non-blocking operations.Make sure to provide adequate comments and documentation in the code.



In [105]:
%%writefile sol08pr02.py

from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

p = 1 # initialize to 1

if rank == 1:
    y = comm.recv(source=0)
    for i in range(5):
        p = comm.recv(source=0)
#         print("Process", rank, "received the number", p, "from process 0")
        p /= y
        comm.send(p, dest=0)
        
if rank == 0:
    for line in sys.stdin:
        x = float(line)
        if x != 0.0:
            break
    for line in sys.stdin:
        y = float(line)
        if y != 0.0:
            break
    comm.send(y, dest=1)
    for i in range(5):
        p *= x
        comm.send(p, dest=1)
        p = comm.recv(source=1)
#         print("Process", rank, "received the number", p, "from process 1")
    print("Process", rank, "received the number", p, "from process 1")

Overwriting sol08pr02.py
